- 교제 정보 : https://goo.gl/eFk5uN

# Lesson1. Extended Relational Algebra

----

## Topic1. Extended Relational Algebra Operation

### 1-1. Aggregation: GROUP BY

영역별로 그룹을 지어서 그 중에서 계산된 값을 반환한다.
$$ _{Branch}G_{max(Balance)}(Bank) $$

- 예 : Bank라는 테이블을 만든뒤 각 브랜치별로 그룹을 지어서 그 중의 최대 잔고를 조회하는 예
- 식 : **SELECT Branch, max(Balance) FROM Bank GROUP BY Branch**

실습 참고 
- SQLite in Jupyter Notebook : https://docs.python.org/2/library/sqlite3.html
- Install SQLite in Mac : https://goo.gl/ruiZEK

In [1]:
import sqlite3 as sql

In [2]:
# 작성한 테이블 삭제용
conn = sql.connect("Bank.sqlite")
cursor = conn.cursor()
cursor.execute("""DROP TABLE Bank""")

In [3]:
conn = sql.connect("Bank.sqlite")
cursor = conn.cursor()
cursor.execute("""CREATE TABLE Bank(Account int, Branch text, Country text, Balance text) """)
cursor.execute("INSERT INTO Bank(Account, Branch, Country, Balance) VALUES(100090, 'Seoul', 'Korea', 10000)")
cursor.execute("INSERT INTO Bank(Account, Branch, Country, Balance) VALUES(100100, 'Busan', 'Korea', 9000)")
cursor.execute("INSERT INTO Bank(Account, Branch, Country, Balance) VALUES(200010, 'New York', 'USA', 20000)")
cursor.execute("INSERT INTO Bank(Account, Branch, Country, Balance) VALUES(100090, 'Seoul', 'Korea', 3000)")
cursor.execute("INSERT INTO Bank(Account, Branch, Country, Balance) VALUES(100100, 'Busan', 'Korea', 4000)")
cursor.execute("INSERT INTO Bank(Account, Branch, Country, Balance) VALUES(200010, 'New York', 'USA', 15000)")
conn.commit()

In [4]:
conn = sql.connect("Bank.sqlite")
cursor = conn.cursor()
for row in cursor.execute("SELECT Branch, max(Balance) FROM Bank GROUP BY Branch"):
    print(row)
cursor.close()

('Busan', '9000')
('New York', '20000')
('Seoul', '3000')


### 1-2. Aggregation: GROUP BY ... HAVING ....

그룹별로 정렬한 것 중에서 조건식을 적용할 수 있다.

$$ \delta_{sum(Balance)>10,000}(_{Branch}G_{Balance}(Bank)) $$

- 예: 각 브랜치별로 정렬한 후 최고 잔액을 구하여, 그 중 10,0000 이상인 것만 조회
- HAVING이라는 조건이 추가. SELECT에 있는 요소가 와야 한다

In [5]:
conn = sql.connect("Bank.sqlite")
cursor = conn.cursor()
for row in cursor.execute("""
SELECT Branch, sum(Balance) 
FROM Bank 
GROUP BY Branch 
HAVING sum(Balance) > 10000"):
    print(row)
cursor.close()

('Busan', 13000)
('New York', 35000)
('Seoul', 13000)


### 1-3. Aggregation: ORDER BY

내림차순 오름차순 정렬 기능을 적용할 수 있다.

$$ \delta_{sum(Balance)>10,000}(_{Branch}G_{Balance}(\delta_{countty=korea}Bank)) $$

- 예: 각 브랜치별로 정렬한 후 최고 잔액을 구하여, 그 중 10,0000 이상인 것과 지역이 한국인 것을 구하여 이를 오름차순으로 정렬
- ORDER BY를 사용하며 ASC는 오름차순, DSC는 내림차순 정렬
- Sets는 중복불가, Bags는 중복허용 집합. 둘다 정렬을 고려하지 않기 때문에 해당 기능을 사용

----

## Topic2. Interpreting Complicated SQL

### 2-1. Interpreting Complicated SQL 1

복잡한 쿼리문을 보면서 어떻게 조회되는지 해석해보자.
- 서브쿼리 : 쿼리 안에 또 다른 쿼리가 들어가는 것
- 제일 안에 있는 쿼리부터 봐야 한다
- 먼저 Tokyo_4_merged_date_time 테이블에 ts, binsize 칼럼이 추가됨
- 그 뒤에 연산값을 binid로 저장하여 새로운 칼럼으로 또 추가됨
- avg : 평균을 구하는 함수

``` mysql
SELECT binid,
    round(avg(cast(Fluo as float)),3) as Fluo,
    round(avg(cast(Oxygen as float)),3) as Oxygen,
    round(avg(cast(Nitrate)uM as float)),3) as Nitrate_uM,
    round(avg(cast(longitude as float)),3) as longitude,
    round(avg(cast(latitude as float)),3) as latitude,
FROM (
        SELECT *,
        cast( floor(ts) + floor((ts - floor(ts)*24*60/binsize) * binsize / (24*60) as datetime) as binid
        FROM(
            SELECT *,
                cast(timestamp as float) as ts, 
                5.0 as binsize
            FROM Tokyo_4_merged_date_time
        ) x
    ) bins
GROUP BY binid
ORDER BY binid ASC

```

### 2-1. Interpreting Complicated SQL 2

이번에는 SELECT가 복잡한 예를 살펴보자. CASE WHEN이라는 조건이 포함되었는데 경우에 따라 값을 다르게 설정할 때 사용한다.
- SELECT 문의 요소는 총 9개가 만들어 짐
- CASE WHEN 구절이 복잡하므로 사용자 함수로 줄여쓸 수 있다

#### User Define 함수 적용 전

``` mysql
SELECT x.strain, x.chr, x.regionas snp_region, x.start_bpas snp_start_bp, x.end_bpas snp_end_bp, w.start_bpas nc_start_bp, w.end_bpas nc_end_bp, w.categoryas nc_category, 
    CASE WHEN (x.start_bp >= w.start_bp AND x.end_bp <= w.end_bp)
        THEN x.end_bp-x.start_bp+ 1
    WHEN (x.start_bp <= w.start_bp AND w.start_bp <= x.end_bp)
        THEN x.end_bp-w.start_bp+ 1
    WHEN (x.start_bp <= w.end_bp AND w.end_bp <= x.end_bp)
        THEN w.end_bp-x.start_bp+ 1
    END AS len_overlap
FROM hotspots_deserts x JOIN table_noncoding_positions w ON x.chr = w.chr
WHERE (x.start_bp>= w.start_bpANDx.end_bp<= w.end_bp) 
    OR (x.start_bp<= w.start_bpANDw.start_bp<= x.end_bp)
    OR (x.start_bp<= w.end_bpANDw.end_bp<= x.end_bp)
ORDER BY x.strain, x.chrASC,x.start_bpASC

```

#### User Define 함수 적용 후
len_overlap 구문으로 교체한다

``` mysql
SELECT x.strain, x.chr, x.regionas snp_region, x.start_bpas snp_start_bp, x.end_bpas snp_end_bp, w.start_bpas nc_start_bp, w.end_bpas nc_end_bp, w.categoryas nc_category, 
    len_overlap(x.start_bp, x.end_bp, w.start_bp, w.end_bp)
FROM hotspots_deserts x JOIN table_noncoding_positions w ON x.chr = w.chr
WHERE overlaps(x.start_bp, x.end_bp, w.start_bp, w.end_bp)
ORDER BY x.strain, x.chrASC,x.start_bpASC

```

#### 최종수정 후
FROM의 조인 구문도 줄일 수 있다. JOIN의 조건을 WHERE 절에 사용한다.

``` mysql
SELECT x.strain, x.chr, x.regionas snp_region, x.start_bpas snp_start_bp, x.end_bpas snp_end_bp, w.start_bpas nc_start_bp, w.end_bpas nc_end_bp, w.categoryas nc_category, 
    len_overlap(x.start_bp, x.end_bp, w.start_bp, w.end_bp)
FROM hotspots_deserts x, table_noncoding_positions w
WHERE x.chr = w.chr AND overlaps(x.start_bp, x.end_bp, w.start_bp, w.end_bp)
ORDER BY x.strain, x.chrASC,x.start_bpASC

```

----

## Topic3. User-Defined Function

### 3-1. User Difined Function

사용자가 직접 함수를 작성하여 등록할 수 있다. 많은 데이터베이스 소프트웨어에서 지원하는 기능이지만 SQLite에서는 지원하지 않는다.

1. **Scalar Function** : 스칼라를 반환
1. **Aggregate Function** : 통합값을 반환
1. **Table Function** : 테이블을 반환

----